In [2]:
#library

import pandas as pd
import contractions
import re
import string
from cleantext import clean
from nltk.tokenize import word_tokenize
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet

In [3]:
#data cleansing

def cleansing(tweet):
    #hapus RT Tag
    t1 = re.sub('RT\s', '', tweet) 
    #hapus @_username
    t2 = re.sub('\B@\w+', '', t1)
    #hapus emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    
    t3 = emoji_pattern.sub(r'', t2)
    #hapus URL
    t4 = re.sub(r'\S*https?:\S*', '', t3)
    #hapus tagar
    t5 = re.sub('#+', '', t4)
    #low text/case folding
    t6 = t5.lower()
    #alphabets only, exclude numbers and special characters
    t7 = re.sub(r'[^a-zA-Z]', ' ', t6)
    #Replace contractions with their extended forms
    t8 = contractions.fix(t7)
    return t8

In [ ]:
file = input('Enter File Name:') #raw_tweet

df = pd.read_csv('raw_tweets_' + file + '.csv', 
                 usecols=['created_at', 'date', 'user_id', 
                          'username', 'name', 'tweet', 'link'])
df

In [ ]:
for i, r in df.iterrows():
    y = cleansing(r['tweet'])
    df.loc[i, 'tweet'] = y
    
df.head()

In [ ]:
#POS Tagger Dictionary
post_dict = {'J/':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

In [ ]:
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('indonesian')):
            newlist.append(tuple([word, post_dict.get(tag[0])]))
    return newlist

In [ ]:
df['POS tagged'] = df['tweet'].apply(token_stop_pos)
df.head()

In [ ]:
export = input('Enter File Name:') #cleaned_tweet

df.to_csv("cleaned_tweets_" + export + ".csv", index=False)